## Uso di `lxml` con file HTML

In [4]:
from lxml import etree

html_string = '''
<!DOCTYPE html>
<html>
<head>
    <title>Library</title>
</head>
<body>
    <div class="book" id="1">
        <h2>Python Programming</h2>
        <p>Author: John Doe</p>
        <p>Year: 2020</p>
    </div>
    <div class="book" id="2">
        <h2>Learning XML</h2>
        <p>Author: Jane Smith</p>
        <p>Year: 2018</p>
    </div>
</body>
</html>
'''

root = etree.fromstring(html_string)

Osserviamo che i libri sono rappresentati da dei `<div>` aventi la classe `book`.

Per ottenere tutti i libri, possiamo dunque cercare all'interno della pagina, tutti i tag `<div>` con questa caratteristica.

Possiamo usare il metodo `Element.findall()`:

In [5]:
books = root.findall('.//div[@class="book"]')

print(books)

[<Element div at 0x10e072080>, <Element div at 0x10e071f00>]


Oppure il metodo `Element.xpath()`:

In [8]:
books = root.xpath('//div[@class="book"]')

print(books)

[<Element div at 0x10e072080>, <Element div at 0x10e071f00>]


Ora che abbiamo gli elementi che rappresentano i libri, possiamo cliclarli uno per uno ed estrarre le informazioni che ci interessano:

In [26]:
for book in books:

    # L'ID è un attributo del <div> attuale (che è book), quindi lo otteniamo
    # con il metodo .get()
    book_id = book.get('id')

    # Sappiamo che il titolo del libro è contenuto in un tag <h2>, che è unico
    # all'interno del libro. Quindi il metodo .find() è sufficiente.
    title = book.find('h2').text

    # Invece l'autore e l'anno sono contenuti in un tag <p>. Tuttavia di questi
    # tag ne abbiamo due, quindi per ottenerli entrambi, usiamo il metodo .findall()
    paragraphs = book.findall('p')

    # Ora dobbiamo cercare qual è il paragrafo contenente la parola "Author"
    # quello contenente la parola "Year". Dato che queste parole sono all'inizio
    # del testo, possiamo usare il metodo str.startswith()
    for p in paragraphs:
        if p.text.startswith('Author'):
            # Infine facciamo lo split del testo sui due punti (:) e prediamo
            # il secondo elemento della lista che ci viene restituita da .split()
            author = p.text.split(': ')[1]
        elif p.text.startswith('Year'):
            year = p.text.split(': ')[1]

    print(f"Book ID: {book_id}")
    print(f"Title: {title}")
    print(f"Author: {author}")
    print(f"Year: {year}")
    print("-------------------")

Book ID: 1
Title: Python Programming
Author: John Doe
Year: 2020
-------------------
Book ID: 2
Title: Learning XML
Author: Jane Smith
Year: 2018
-------------------


In questo caso, per l'autore e l'anno, il metodo `Element.xpath()` è invece molto più versatile di `Element.findall()`, perché consente di usare predicati e condizioni.

Per esempio, se vogliamo trovare un tag `<p>` che contiene il testo "Author" o "Year"

In [27]:
for book in books:
    book_id = book.get('id')
    title = book.find('h2').text
    
    # Il metodo .xpath() è molto più comodo 
    author_elem = book.xpath('.//p[contains(text(), "Author")]')[0]
    year_elem = book.xpath('.//p[contains(text(), "Year")]')[0]
    
    author = author_elem.text.split(': ')[1]
    year = year_elem.text.split(': ')[1]

    print(f"Book ID: {book_id}")
    print(f"Title: {title}")
    print(f"Author: {author}")
    print(f"Year: {year}")
    print("-------------------")

Book ID: 1
Title: Python Programming
Author: John Doe
Year: 2020
-------------------
Book ID: 2
Title: Learning XML
Author: Jane Smith
Year: 2018
-------------------


### Qual è la differenza tra i metodi `.findall()` e `.xpath()` degli `Element`?

`findall()` fa parte dell'API originale di `ElementTree`. Supporta un semplice sottoinsieme del linguaggio XPath, senza predicati, condizioni e altre caratteristiche avanzate. È molto utile per trovare velocemente tag specifici in un albero.

`xpath()`, invece, supporta tutta la potenza del linguaggio XPath, compresi i predicati, le funzioni XPath e le funzioni di estensione Python. La sintassi è definita dalle specifiche XPath. Se si ha bisogno dell'espressività e della selettività di XPath, il metodo `xpath()` è la scelta migliore.

> PER APPROFONDIRE: [What are the findall() and xpath() methods on Element(Tree)?
](https://lxml.de/FAQ.html#what-are-the-findall-and-xpath-methods-on-element-tree)


### Ulteriori informazioni sulla sintassi XPath

- [XPath con `lxml`](https://lxml.de/xpathxslt.html)
- [Cheatsheet 1](https://devhints.io/xpath)
- [Cheatsheet 2](https://quickref.me/xpath.html)


### Esempi di predicati e condizioni comuni XPath
Ecco alcuni esempi di espressioni XPath con predicati e condizioni comuni che potrebbero esserti utili:

- **Selezionare nodi in base al contenuto del testo**:
  ```xpath
  //p[contains(text(), "Author")]
  ```

- **Selezionare nodi in base a un attributo**:
  ```xpath
  //a[@href='http://example.com']
  ```

- **Selezionare nodi figli specifici**:
  ```xpath
  /root/child::node()
  ```

- **Selezionare nodi con un attributo specifico**:
  ```xpath
  //element[@attribute='value']
  ```

- **Selezionare nodi con più condizioni**:
  ```xpath
  //book[price>30 and @category='fiction']
  ```

- **Utilizzare operatori logici**:
  ```xpath
  //element[@attribute='value1' or @attribute='value2']
  ```

- **Selezionare nodi basati sulla posizione**:
  ```xpath
  //book[position()=1]
  ```

### Esempio pratico con `lxml` e XPath
Per utilizzare questi predicati e condizioni con `lxml` in Python, ecco un esempio:

In [28]:
from lxml import etree

# Esempio XML
xml_data = """
<library>
    <book category="fiction">
        <title lang="en">Harry Potter</title>
        <author>J.K. Rowling</author>
        <price>29.99</price>
    </book>
    <book category="non-fiction">
        <title lang="en">A Brief History of Time</title>
        <author>Stephen Hawking</author>
        <price>15.99</price>
    </book>
</library>
"""

# Parsing dell'XML
root = etree.fromstring(xml_data)

# Esempio di XPath con predicati
books = root.xpath('//book[price>20 and @category="fiction"]')

for book in books:
    print(book.find('title').text)

Harry Potter


In [9]:
from lxml import etree

# Esempio HTML
html_data = """
<!DOCTYPE html>
<html>
<head>
    <title>Library</title>
</head>
<body>
    <div class="book" id="1">
        <h2>Python Programming</h2>
        <p>Author: John Doe</p>
        <p>Year: 2020</p>
    </div>
    <div class="book" id="2">
        <h2>Learning XML</h2>
        <p>Author: Jane Smith</p>
        <p>Year: 2018</p>
    </div>
    <div class="book" id="3">
        <h2>XPath Superpowers</h2>
        <p>Author: Dr. X</p>
        <p>Year: 2020</p>
    </div>
</body>
</html>
"""

# Parsing dell'HTML
root = etree.fromstring(html_data)

# Esempio di XPath con predicati
books = root.xpath('//div[@class="book" and p[contains(text(), "Year: 2020")]]')

for book in books:
    etree.dump(book)

<div class="book" id="1">
        <h2>Python Programming</h2>
        <p>Author: John Doe</p>
        <p>Year: 2020</p>
    </div>
    
<div class="book" id="3">
        <h2>XPath Superpowers</h2>
        <p>Author: Dr. X</p>
        <p>Year: 2020</p>
    </div>



## Introduzione a BeautifulSoup

Nel mondo del *web scraping* con Python, la libreria BeautifulSoup è quella storicamente più usata.

A differenza di `lxml`, BeautifulSoup è dedicato esclusivamente ai file HTML ed utilizza principalmente i [selettori CSS](https://www.w3schools.com/cssref/css_selectors.php) per selezionare gli elementi desiderati all'interno di un documento.

Installare `beautifulsoup4`:

```cmd
> pip install beautifulsoup4

> py -m pip install beautifulsoup4
```

In [10]:
from bs4 import BeautifulSoup

# Dati in HTML
html_data = """
<!DOCTYPE html>
<html>
<head>
    <title>Library</title>
</head>
<body>
    <div class="book" id="1">
        <h2>Python Programming</h2>
        <p>Author: John Doe</p>
        <p>Year: 2020</p>
    </div>
    <div class="book" id="2">
        <h2>Learning XML</h2>
        <p>Author: Jane Smith</p>
        <p>Year: 2018</p>
    </div>
    <div class="book" id="3">
        <h2>XPath Superpowers</h2>
        <p>Author: Dr. X</p>
        <p>Year: 2020</p>
    </div>
</body>
</html>
"""

# Parsing dell'HTML
soup = BeautifulSoup(html_data, 'html.parser')

# Selezione degli elementi div con classe "book"
# NOTA: div.book è un selettore CSS
books = soup.select('div.book')

for book in books:
    paragraphs = book.find_all('p')
    for p in paragraphs:
        if "Year: 2020" in p.text:
            print(book.text)



Python Programming
Author: John Doe
Year: 2020


XPath Superpowers
Author: Dr. X
Year: 2020



L'[esercizio_26_xml_html.ipynb](../../../esercizi/esercizio_26_xml_html.ipynb) possiamo risolverlo anche usando la libreria BeautifulSoup:

In [44]:
from bs4 import BeautifulSoup

# Esempio HTML
html_data = """
<!DOCTYPE html>
<html>
<head>
    <title>Library</title>
</head>
<body>
    <div class="book" id="1">
        <h2>Python Programming</h2>
        <p>Author: John Doe</p>
        <p>Year: 2020</p>
    </div>
    <div class="book" id="2">
        <h2>Learning XML</h2>
        <p>Author: Jane Smith</p>
        <p>Year: 2018</p>
    </div>
</body>
</html>
"""

# Parsing dell'HTML
soup = BeautifulSoup(html_data, 'html.parser')

# Selezione degli elementi div con classe "book"
books = soup.select('div.book')

for book in books:
    book_id = book.get('id')
    title = book.find('h2').text
    
    # Trova l'elemento <p> contenente "Author"
    author_elem = book.find('p', string=lambda x: "Author" in x)
    # Trova l'elemento <p> contenente "Year"
    year_elem = book.find('p', string=lambda x: "Year" in x)
    
    author = author_elem.text.split(': ')[1]
    year = year_elem.text.split(': ')[1]

    print(f"Book ID: {book_id}")
    print(f"Title: {title}")
    print(f"Author: {author}")
    print(f"Year: {year}")
    print("-------------------")


Book ID: 1
Title: Python Programming
Author: John Doe
Year: 2020
-------------------
Book ID: 2
Title: Learning XML
Author: Jane Smith
Year: 2018
-------------------


## Compiti per la prossima lezione

Dare una lettura ai seguenti notebook:

- [12_python_http.ipynb](../../../12_python_http.ipynb) fino al paragrafo "Parametri delle *GET query string*"

- [13_python_spider.ipynb](../../../13_python_spider.ipynb) dal paragrafo "Request + Beautiful Soup" in avanti.
